In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import string
import nltk
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

np.random.seed(10)

# Описание задачи

Анализ текста на основе отзывов с сайта IMDB.

Загрузим датасет и ознакомимся с ним. Исходный датасет взят с сайта Kaggle: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [2]:
data = pd.read_csv('IMDB Dataset.csv', sep = ',', decimal = '.', encoding = 'utf-8')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Описание столбцов

- review - Отзыв пользователя.
- sentiment - Тип отзыва.

Посмотрим какие значения принимает тип отзыва

In [3]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

Как мы видим, половина отзывов у нас размечены как позитивные, остальная часть как негативные. Рассмотрим первый отзыв в наших данных и выведем его тип

In [4]:
print(data.loc[0, 'review'])
print('\nsentiment = ', data.loc[0, 'sentiment'])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

Из самого отзыва и его разметки мы видим, что он является положительным.
Теперь заменим символ "br" на обычный пробел.

In [5]:
data['review'] = data['review'].str.replace('<br />', ' ')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming t...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Обработка текста
Обработаем текст наших отзывов:
- исключим стоп-слова.
- уберем знаки препинания.
- применим леммитизацию.

In [6]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

# примеры стоп-слов
stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Евгений\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [7]:
word_tokenizer = nltk.WordPunctTokenizer()

In [8]:
dates = [str(x) for x in np.arange(1900, 2022)]
def process_data(data):
    texts = []
    targets = []
    
    for i in range(len(data)):
               
        text_lower = data.loc[i, 'review'].lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем екст на слова
        
        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and word not in dates)]
        
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [9]:
y = data['sentiment'].values
texts = process_data(data)

In [10]:
# Пример обработанного первого отзыва
i = 0
print("Label: ", y[i])
print("Tokens: ", texts[i][:5])

Label:  positive
Tokens:  ['one', 'reviewers', 'mentioned', 'watching', '1']


In [11]:
# инициализируем стеммер
stemmer = SnowballStemmer('english')

In [12]:
# применяем лемматизацию ко всем текстам
for i in tqdm_notebook(range(len(texts))):
    text_lemmatized = [stemmer.stem(x) for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts[i] = ' '.join(text_lemmatized)                # объединяем все слова в одну строку через пробел

In [13]:
i = 0
print("Label: ",   y[i])
print("Text: \n",  texts[i])

Label:  positive
Text: 
 one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani .. aryan muslim gangsta latino christian italian irish .... scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc ... oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view .

# Преобразование текста в вектора на основе TFiDF и построение модели
Для начала разобьем наш датасет на train и test выборки.

In [14]:
def label2num(y):
    if y == 'positive':
        return 1
    if y == 'negative':
        return 0

encoded_y = [label2num(yy) for yy in y]

In [15]:
train_texts, test_texts, train_y, test_y = train_test_split(texts, encoded_y, test_size=0.2, random_state=42, stratify = y)

Запишем функцию для сбора основных метрик по построенным моделям

In [17]:
def quality_report(prediction, prediction_prob, actual, model):
    quality_data = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'f1_score', 'ROC-AUC'], index=[model])
    
    quality_data['Accuracy'] = round(accuracy_score(prediction, actual), 3)
    quality_data['Precision'] = round(precision_score(prediction, actual), 3)
    quality_data['Recall'] = round(recall_score(prediction, actual), 3)
    quality_data['f1_score'] = round(f1_score(prediction, actual), 3)
    quality_data['ROC-AUC'] = round(roc_auc_score(actual, prediction_prob), 3)
    
    return quality_data

Зададим параметры регуляризации логистической регрессии, которые будем подбирать с помощью GridSearch.

In [18]:
parameters = {'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
             'penalty':['none', 'elasticnet', 'l1', 'l2']}

Зададим список для перебора параметра max_features в векторайзере. Т.е. сколько наиболее встречающихся слов мы будем учитывать при построении векторов.

In [19]:
max_features_list = [100, 200, 300, 500, 1000, 5000, 10000, 20000]

Запишем цикл с перебором параметра max_features, построением векторайзера, построения логистической регрессии и сбора метрик по модели.

In [21]:
report_final = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'f1_score', 'ROC-AUC'])
params_final = pd.DataFrame(columns=['C', 'penalty'])

for i in tqdm_notebook(max_features_list):
    vectorizer = TfidfVectorizer(max_features = i, norm = None)
    train_X = vectorizer.fit_transform(train_texts)
    test_X  = vectorizer.transform(test_texts)
    
    logit = LogisticRegression()
    logit_gs = GridSearchCV(logit,  
                      parameters,
                      scoring='accuracy',
                      cv=5) 
    logit_gs.fit(train_X, train_y)
    params_data = pd.DataFrame(columns=['C', 'penalty'], index=['Vectorizer Max Features - ' + str(i)])
    params_data['C'] = logit_gs.best_params_['C']
    params_data['penalty'] = logit_gs.best_params_['penalty']
    params_final = pd.concat((params_final, params_data), axis=0)
    logit_pred = logit_gs.predict(test_X)
    logit_pred_prob = logit_gs.predict_proba(test_X)[:, 1]
    logit_report = quality_report(logit_pred, logit_pred_prob, test_y, 'Vectorizer Max Features - ' + str(i))
    
    report_final = pd.concat((report_final, logit_report), axis=0)

Посмотрим на полученные метрики по моделям.

In [22]:
report_final

,Accuracy,Precision,Recall,f1_score,ROC-AUC
Vectorizer Max Features - 100,0.751,0.751,0.751,0.751,0.823
Vectorizer Max Features - 200,0.802,0.814,0.795,0.804,0.877
Vectorizer Max Features - 300,0.830,0.845,0.821,0.833,0.905
Vectorizer Max Features - 500,0.852,0.862,0.845,0.854,0.924
Vectorizer Max Features - 1000,0.867,0.877,0.860,0.868,0.936
Vectorizer Max Features - 5000,0.887,0.895,0.881,0.888,0.950
Vectorizer Max Features - 10000,0.893,0.899,0.887,0.893,0.954
Vectorizer Max Features - 20000,0.895,0.902,0.890,0.896,0.956


Как мы видим, чем больше слов мы передаем в векторайзер, тем наша модель становится лучше. Также можно отметить, что при 10000 и 20000 слов, мы получаем практически схожие результаты. По метрикам можно сделать вывод, что мы получили очень хорошую модель предсказания типа отзыва к фильму.

Теперь построим финальную модель с учетом выбранных параметров. Посмотрим на подобранные параметры регуляризации.

In [23]:
params_final

,C,penalty
Vectorizer Max Features - 100,0.001,l2
Vectorizer Max Features - 200,0.001,l2
Vectorizer Max Features - 300,0.010,l2
Vectorizer Max Features - 500,0.001,l2
Vectorizer Max Features - 1000,0.001,l2
Vectorizer Max Features - 5000,0.001,l2
Vectorizer Max Features - 10000,0.001,l2
Vectorizer Max Features - 20000,0.001,l2


Построим модель с выбранными параметрами. Параметры в векторайзере уже подобраны верные, так как в цикле мы пришли к 20 000.

In [28]:
final_model = LogisticRegression(C=0.001, penalty='l2').fit(train_X, train_y)

Сделаем предсказание для тестовых данных

In [29]:
model_pred = final_model.predict(test_X)

Запишем функцию для декодирования целевой переменной и предсказанных значений.

In [30]:
def num2label(y):
    if y == 1:
        return 'positive'
    if y == 0:
        return 'negative'
    
decoded_pred = [num2label(y) for y in model_pred]
decoded_test_y = [num2label(y) for y in test_y]

Посмотрим на несколько примеров.

In [33]:
for i in range(5):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст отзыва: ', test_texts[i][:500]+'...')
    print('\n')

Истинный лейбл: negative
Предсказанный лейбл: positive
Текст отзыва:  yes mtv realli way market daria start clever teenag angst -" comment everyth suck make viewer feel better sucki teenag life sitcom mutat deal problem "- charad use watch daria time love sit watch call movi wonder point daria tell us lead life colleg excus point daria made everi episod like ok long ok matter rest sick sad world think entir thing remind scene realiti bite movi channel show documentri first time...


Истинный лейбл: negative
Предсказанный лейбл: negative
Текст отзыва:  stori bride fair amus engag one filmmak credit set portray rural minnesotan respect ordinarili reserv coast dweller weird though find independ movi brainchild singl person unambiti cliché ridden committe brew hollywood potboil portrait rural peopl intend affection think charact ring true -- quit meal small town diner never overheard debat merit differ nineteenth centuri english novelist one might suggest writer director seman experi rural

Как мы видим, из 5 отзывов к фильмов наша модель верно предсказала значения для 4 отзывов.